# 2 Easy Wins
##### **Author: Adam Gatt**

Later notebooks will cover more advance topics but this one aims to show off simple new language features that you should be able to immediately benefit from with little/no trade-off. Many revolve around enforcing correctness, to help remove some classes of bugs before they have a chance to happen.

The [clang-tidy](https://clang.llvm.org/extra/clang-tidy/) linter will be able to make suggestions about many of these language features, pointing out situation where they can be used. With some care and manual review, you can even run the linter over an entire existing codebase and have the suggestions applied automatically.

## override

How come the duck below fails to realise it can quack?

In [1]:
#include <iostream>

class Bird {
    public:
    virtual void makeNoise() const {
        std::cout << "Chirp!" << std::endl;
    }
};

class Duck : public Bird {
    public:
    virtual void makeNoise() {
        std:: cout << "Quack!" << std::endl;
    }
};

Bird* daffy = new Duck();

daffy->makeNoise();

Chirp!


It's because the two `makeNoise()` functions aren't the same! Because one is const and the other isn't, they count as having different signatures. This means the derived class isn't overriding the makeNoise in the base class; the two methods don't occupy the same entry in the vtable. 

Constness is a subtle way this bug can occur, but it can also be caused by a typo in the method name or differences in parameter types.

Because both the base and overridden methods use the same syntax (beginning `virtual` annotation), the compiler doesn't have a clear way to know that `Duck::makeNoise` is meant to be an overridden method instead of starting its own base virtual method. `override` is a new annotation to add to the overriding method and make it clear to the compiler. When a method is marked `override` the compiler will search for a matching base virtual method to ensure it exists, and will throw a compilation error if one cannot be found.

As an added benefit it also makes the intention clear to the human reader.

In [2]:
class Turkey : public Bird {
    public:
    virtual void makeNoise() override {
        std:: cout << "Gobble!" << std::endl;
    }
};

input_line_9:3:18: error: 'makeNoise' marked 'override' but does not override any member functions
    virtual void makeNoise() override {
                 ^


Interpreter Error: 

## nullptr

`nullptr` is a Modern C++ keywork for the null reference, where in the past you might have used `0` or `NULL` (often #defined to be 0). The advantage of nullptr is that it has its own type (`std::nullptr_t`), whereas the `int` type of NULL can allow for subtle bugs or ambiguities, especially in constructors and overloaded functions. In the best-case scenario the compiler will notice the ambiguity with a warning or error, in the worst-case you may have a subtle logic error.

In [3]:
class Colour {    
    public:
    int rgbCode;
    
    // Specify the rgb code directly
    Colour(int rgbCode) : rgbCode(rgbCode) { }
    
    // Copy the rgb code from another Colour
    Colour(const Colour* cloneSrc) {
        if (cloneSrc != NULL) {
            rgbCode = cloneSrc->rgbCode;
        }
    }
}

In [4]:
// Which constructor will be called?

Colour col1(NULL);

input_line_11:3:8: error: call to constructor of '__cling_N54::Colour' is ambiguous
Colour col1(NULL);
       ^    ~~~~
input_line_10:6:5: note: candidate constructor
    Colour(int rgbCode) : rgbCode(rgbCode) { }
    ^
input_line_10:9:5: note: candidate constructor
    Colour(const Colour* cloneSrc) {
    ^
input_line_10:1:7: note: candidate is the implicit copy constructor
class Colour {    
      ^
input_line_10:1:7: note: candidate is the implicit move constructor


Interpreter Error: 

In [5]:
Colour col2(nullptr);

Notice the constructor executing correctly instead of trying to dereference nullptr. This is because the guard clause `if (cloneSrc != NULL)` still performed its job. The new keyword enjoys strong interoperability with existing legacy code as the rules for comparing nullptr and NULL have been well-defined to produce intuitive results.

In [6]:
bool comparisons[] {
    nullptr == nullptr,
    nullptr != nullptr,
    nullptr == NULL,
    nullptr != NULL,
    nullptr == 0,
    nullptr != 0,
    nullptr ? true : false
};

comparisons

input_line_13:5:13: warning: comparison between NULL and non-pointer ('nullptr_t' and NULL) [-Wnull-arithmetic]
    nullptr == NULL,
    ~~~~~~~ ^  ~~~~
input_line_13:6:13: warning: comparison between NULL and non-pointer ('nullptr_t' and NULL) [-Wnull-arithmetic]
    nullptr != NULL,
    ~~~~~~~ ^  ~~~~
input_line_13:9:5: warning: implicit conversion of nullptr constant to 'bool' [-Wnull-conversion]
    nullptr ? true : false
    ^~~~~~~ ~
    false
In file included from input_line_1:1:
In file included from /home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/new:40:
In file included from /home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/exception:144:
In file included from /home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/bits/nested_exception.h:40:
In file included from

/home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/ostream:685:24: note: in instantiation of template class 'std::__and_<std::__not_<std::is_lvalue_reference<std::basic_ostream<char> &> >,
      std::__is_convertible_to_basic_ostream<std::basic_ostream<char> &>, std::__is_insertable<std::basic_ostream<char> &, const std::complex<double> &, void> >' requested
      here
    typename enable_if<__and_<__not_<is_lvalue_reference<_Ostream>>,
                       ^
/home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/complex:1531:28: note: while substituting deduced template arguments into function template 'operator<<' [with _Ostream = std::basic_ostream<char> &, _Tp = std::complex<double>]
  extern template ostream& operator<<(ostream&, const complex<double>&);
                           ^
In file included from input_line_1:1:
In file included from

/home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/type_traits:137:31: error: no member named 'value' in 'std::__not_<std::is_lvalue_reference<std::basic_ostream<wchar_t> &> >'
    : public conditional<_B1::value, __and_<_B2, _B3, _Bn...>, _B1>::type
                         ~~~~~^
/home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/ostream:685:24: note: in instantiation of template class 'std::__and_<std::__not_<std::is_lvalue_reference<std::basic_ostream<wchar_t> &> >,
      std::__is_convertible_to_basic_ostream<std::basic_ostream<wchar_t> &>, std::__is_insertable<std::basic_ostream<wchar_t> &, const std::complex<float> &, void> >'
      requested here
    typename enable_if<__and_<__not_<is_lvalue_reference<_Ostream>>,
                       ^
/home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linu

In file included from /home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/bits/nested_exception.h:40:
In file included from /home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/bits/move.h:55:
/home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/type_traits:137:31: error: no member named 'value' in 'std::__not_<std::is_lvalue_reference<std::basic_ostream<wchar_t> &> >'
    : public conditional<_B1::value, __and_<_B2, _B3, _Bn...>, _B1>::type
                         ~~~~~^
/home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/ostream:685:24: note: in instantiation of template class 'std::__and_<std::__not_<std::is_lvalue_reference<std::basic_ostream<wchar_t> &> >,
      std::__is_convertible_to_basic_ostream<st

      char, std::allocator<unsigned char> > > > > >, std::is_reference<std::_Rb_tree_iterator<std::pair<const std::__cxx11::basic_string<char>,
      nlohmann::basic_json<std::map, std::vector, std::__cxx11::basic_string<char>, bool, long, unsigned long, double, std::allocator, adl_serializer, std::vector<unsigned
      char, std::allocator<unsigned char> > > > > >, std::is_void<std::_Rb_tree_iterator<std::pair<const std::__cxx11::basic_string<char>, nlohmann::basic_json<std::map,
      std::vector, std::__cxx11::basic_string<char>, bool, long, unsigned long, double, std::allocator, adl_serializer, std::vector<unsigned char, std::allocator<unsigned
      char> > > > > > > >' requested here
    : public __not_<__or_<is_function<_Tp>, is_reference<_Tp>,
             ^
/home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/type_traits:109:26: note: in instantiation of template class 'std::is_object<std::_Rb_tree_iterator<

      std::vector, std::__cxx11::basic_string<char>, bool, long, unsigned long, double, std::allocator, adl_serializer, std::vector<unsigned char, std::allocator<unsigned
      char> > > > > >' requested here
    : public conditional<_B1::value, _B2, _B1>::type
                         ^
/home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/bits/stl_pair.h:390:3: note: in instantiation of template class 'std::__and_<std::is_move_assignable<std::_Rb_tree_iterator<std::pair<const std::__cxx11::basic_string<char>,
      nlohmann::basic_json<std::map, std::vector, std::__cxx11::basic_string<char>, bool, long, unsigned long, double, std::allocator, adl_serializer, std::vector<unsigned
      char, std::allocator<unsigned char> > > > > >, std::is_move_assignable<bool> >' requested here
                __and_<is_move_assignable<_T1>,
                ^
/home/adamgatt/miniconda3/include/nlohmann/json.hpp:18124:33: note: in inst

        return __and_<is_convertible<const _U1&, _T1>,
               ^
/home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/bits/stl_pair.h:258:7: note: in instantiation of function template specialization 'std::_PCC<true, std::_Rb_tree_iterator<std::pair<const std::__cxx11::basic_string<char>,
      nlohmann::basic_json<std::map, std::vector, std::__cxx11::basic_string<char>, bool, long, unsigned long, double, std::allocator, adl_serializer, std::vector<unsigned
      char, std::allocator<unsigned char> > > > >, bool>::_ImplicitlyConvertiblePair<std::_Rb_tree_iterator<std::pair<const std::__cxx11::basic_string<char>,
      nlohmann::basic_json<std::map, std::vector, std::__cxx11::basic_string<char>, bool, long, unsigned long, double, std::allocator, adl_serializer, std::vector<unsigned
      char, std::allocator<unsigned char> > > > >, bool>' requested here
                           _ImplicitlyConvertiblePair<_U1,

/home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/type_traits:1336:14: note: in instantiation of default argument for '__is_convertible_helper<std::_Rb_tree_iterator<std::pair<const std::__cxx11::basic_string<char>, nlohmann::basic_json<std::map,
      std::vector, std::__cxx11::basic_string<char>, bool, long, unsigned long, double, std::allocator, adl_serializer, std::vector<unsigned char, std::allocator<unsigned
      char> > > > > &&, std::_Rb_tree_iterator<std::pair<const std::__cxx11::basic_string<char>, nlohmann::basic_json<std::map, std::vector,
      std::__cxx11::basic_string<char>, bool, long, unsigned long, double, std::allocator, adl_serializer, std::vector<unsigned char, std::allocator<unsigned char> > > > >
      >' required here
    : public __is_convertible_helper<_From, _To>::type
             ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0

In file included from /home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/new:40:
In file included from /home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/exception:144:
In file included from /home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/bits/nested_exception.h:40:
In file included from /home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/bits/move.h:55:
/home/adamgatt/miniconda3/bin/../lib/gcc/x86_64-conda-linux-gnu/9.3.0/../../../../x86_64-conda-linux-gnu/include/c++/9.3.0/type_traits:142:41: error: no member named 'value' in 'std::__and_<std::is_convertible<std::_Rb_tree_iterator<std::pair<const std::__cxx11::basic_string<char>, nlohmann::basic_json<std::map,
      std::vector, std::__cxx11::basic_

      std::__cxx11::basic_string<char>, bool, long, unsigned long, double, std::allocator, adl_serializer, std::vector<unsigned char, std::allocator<unsigned char> > > > >,
      std::less<void>, std::allocator<std::pair<const std::__cxx11::basic_string<char>, nlohmann::basic_json<std::map, std::vector, std::__cxx11::basic_string<char>, bool,
      long, unsigned long, double, std::allocator, adl_serializer, std::vector<unsigned char, std::allocator<unsigned char> > > > >
      >::_M_emplace_unique<std::__cxx11::basic_string<char>, nlohmann::basic_json<std::map, std::vector, std::__cxx11::basic_string<char>, bool, long, unsigned long,
      double, std::allocator, adl_serializer, std::vector<unsigned char, std::allocator<unsigned char> > > >' requested here
        { return _M_t._M_emplace_unique(std::forward<_Args>(__args)...); }
                      ^
/home/adamgatt/miniconda3/include/nlohmann/json.hpp:18124:33: note: in instantiation of function template specialization 'std::map<st

## explicit (pre-11)

`explicit` 

In [1]:
class Factor {
    public:
    Factor(double value)
        : value(value) { }
    
    double apply(double input) {
        return input * value;
    }
    
    private:
    double value;
}

In [2]:
Factor myFactor = 1/5;

myFactor.apply(100)

0.0000000

In [3]:
class Factor {
    public:
    // Explicit, no implicit type conversion for copy-initialisation
    explicit Factor(double value)
        : value(value) { }
    
    double apply(double input) {
        return input * value;
    }
    
    private:
    double value;
};

Factor myFactor = 1/5;

myFactor.apply(100)

input_line_12:15:9: error: no viable conversion from 'int' to '__cling_N55::Factor'
 Factor myFactor = 1/5;
        ^          ~~~
input_line_12:1:7: note: candidate constructor (the implicit copy constructor) not viable: no known conversion from 'int' to 'const __cling_N55::Factor &' for 1st argument
class Factor {
      ^
input_line_12:1:7: note: candidate constructor (the implicit move constructor) not viable: no known conversion from 'int' to '__cling_N55::Factor &&' for 1st argument
class Factor {
      ^


Interpreter Error: 

## Scoped enums

## using

## auto (in some situations)

## Const references (pre-11)

Despite introducing the useful `nullptr` above, the null reference has been described as its creator as the [Billion Dollar Mistake](https://www.youtube.com/watch?v=YYkOWzrO3xg).